<a href="https://colab.research.google.com/github/hieu1999210/Insincere-question-classification/blob/master/group_Relu_Keras_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cd /content/drive

In [0]:
path_train = "drive/My Drive/data/train.csv"
path_test = "drive/My Drive/data/test.csv"
train = pd.read_csv(path_train)
test = pd.read_csv(path_test)
# Dataset is now stored in a Pandas Dataframe

In [0]:
print(train.shape)
print(test.shape)

(1306122, 3)
(375806, 2)


In [0]:
train, val = train_test_split(train, test_size = 0.1, random_state = 2019) 

In [0]:
print(train.shape)

(1175509, 3)


In [0]:
embed_size = 300
max_features = 50000
maxlen = 100

In [0]:
## fill up the missing values
train_X = train["question_text"].fillna("_na_").values
val_X = val["question_text"].fillna("_na_").values
test_X = test["question_text"].fillna("_na_").values

#Tokenize the sentences
tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

In [0]:
#padding
train_X = pad_sequences(train_X, maxlen = maxlen)
val_X = pad_sequences(val_X, maxlen = maxlen)
test_X = pad_sequences(test_X, maxlen = maxlen)


In [0]:
train_y = train['target'].values
val_y = val['target'].values

In [0]:
print(train_y.shape)
print(val_y.shape)

(1175509,)
(130613,)


In [0]:
# inp = Input(shape = (maxlen,))
# x = Embedding(max_features, embed_size)(inp)
# x = Dropout(0.2)(x)
# x = Conv1D(64, 5, activation = 'relu')(x)
# x = GlobalMaxPool1D()(x)
# x = LSTM(300)(x)
# x = Dense(1, activation = 'sigmoid')

# model = Model(inputs=inp, outputs = x)
# model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
# model.summary()


# x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
# x = GlobalMaxPool1D()(x)
# x = Dense(16,activation="relu")(x)
# x = Dropout(0.1)(x)
# x = Dense(1, activation = "sigmoid")(x)

# model = Model(inputs=inp, outputs = x)
# model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

# model.summary()

from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
model = Sequential()
model.add(Embedding(50000, 300, input_length=100, trainable=True))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(300))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 96, 64)            96064     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 300)               438000    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 301       
Total params: 15,534,365
Trainable params: 15,534,365
Non-trainable params: 0
________________________________________________________________

In [0]:
model.fit(train_X, train_y, batch_size = 512, epochs=5, validation_data=(val_X, val_y))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/5
1175509/1175509 [==============================] - 328s 279us/step - loss: 0.1126 - acc: 0.9553 - val_loss: 0.1114 - val_acc: 0.9556
Epoch 2/5
1175509/1175509 [==============================] - 328s 279us/step - loss: 0.0946 - acc: 0.9615 - val_loss: 0.1139 - val_acc: 0.9559
Epoch 3/5
1175509/1175509 [==============================] - 328s 279us/step - loss: 0.0747 - acc: 0.9701 - val_loss: 0.1288 - val_acc: 0.9526
Epoch 4/5
1175509/1175509 [==============================] - 328s 279us/step - loss: 0.0542 - acc: 0.9787 - val_loss: 0.1468 - val_acc: 0.9509
Epoch 5/5
1175509/1175509 [==============================] - 329s 280us/step - loss: 0.0401 - acc: 0.9845 - val_loss: 0.1762 - val_acc: 0.9505


In [0]:
predict_val_y = model.predict([val_X], batch_size = 1024, verbose = 1)
model.save_weights("Naive_weights.h5")

130613/130613 [==============================] - 8s 58us/step


In [0]:
for thresh in np.arange(0.1, 0.901, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (predict_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.6011468461730242
F1 score at threshold 0.11 is 0.6025155326564631
F1 score at threshold 0.12 is 0.6027984884077213
F1 score at threshold 0.13 is 0.6035203633923502
F1 score at threshold 0.14 is 0.6034311936173541
F1 score at threshold 0.15 is 0.6046903820816865
F1 score at threshold 0.16 is 0.6055387232233029
F1 score at threshold 0.17 is 0.6066645237329905
F1 score at threshold 0.18 is 0.6080380293863439
F1 score at threshold 0.19 is 0.6084399673291588
F1 score at threshold 0.2 is 0.6085001370989854
F1 score at threshold 0.21 is 0.6081431965084803
F1 score at threshold 0.22 is 0.6078300522744968
F1 score at threshold 0.23 is 0.6080188150968753
F1 score at threshold 0.24 is 0.6077834179357022
F1 score at threshold 0.25 is 0.60657319634444
F1 score at threshold 0.26 is 0.6056973226009019
F1 score at threshold 0.27 is 0.6063078072039984
F1 score at threshold 0.28 is 0.6060676105171916
F1 score at threshold 0.29 is 0.6054267619545639
F1 score at threshold 0.